# **Mini Project 02**- W.M.NIPUNI SANDUNIKA (199984810186)

In [23]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Embedding,LSTM
from keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error

**Setting up the data folder into the Colab notebook**

In [24]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
os.chdir("/content/drive/My Drive/Deep Learning/Mini Project 02/")
os.getcwd()

'/content/drive/My Drive/Deep Learning/Mini Project 02'

**Reading the data file**

In [26]:
df = pd.read_csv("CSE.csv")

In [27]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-07-01,798.500000,798.500000,798.500000,798.500000,798.500000,0.0
1,1997-07-02,805.299988,805.299988,805.299988,805.299988,805.299988,0.0
2,1997-07-03,810.799988,810.799988,810.799988,810.799988,810.799988,0.0
3,1997-07-07,828.200012,828.200012,828.200012,828.200012,828.200012,0.0
4,1997-07-08,821.400024,821.400024,821.400024,821.400024,821.400024,0.0
5,1997-07-09,816.700012,816.700012,816.700012,816.700012,816.700012,0.0
6,1997-07-10,820.700012,820.700012,820.700012,820.700012,820.700012,0.0
7,1997-07-11,823.400024,823.400024,823.400024,823.400024,823.400024,0.0
8,1997-07-14,821.900024,821.900024,821.900024,821.900024,821.900024,0.0
9,1997-07-15,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
x = np.array(df["Open"].values)

In [29]:
y = np.array(df["Adj Close"].values)

**Check for missing values in x and y columns**

In [30]:
missing_x = df["Open"].isnull().sum()
missing_y = df["Adj Close"].isnull().sum()

if missing_x > 0 or missing_y > 0:
    print(f"Missing values in 'Open' column: {missing_x}")
    print(f"Missing values in 'Adj Close' column: {missing_y}")

    # Impute missing values with column means
    df["Open"].fillna(df["Open"].mean(), inplace=True)
    df["Adj Close"].fillna(df["Adj Close"].mean(), inplace=True)

    # Verify that missing values are filled
    print("After handling missing values:")
    print(f"Missing values in 'Open' column: {df['Open'].isnull().sum()}")
    print(f"Missing values in 'Adj Close' column: {df['Adj Close'].isnull().sum()}")

# Proceed with the rest code
x = np.array(df["Open"].values)
y = np.array(df["Adj Close"].values)


Missing values in 'Open' column: 439
Missing values in 'Adj Close' column: 439
After handling missing values:
Missing values in 'Open' column: 0
Missing values in 'Adj Close' column: 0


**Spliting the data set into train and test**

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2)

**Model Creation and Calculate RMSE values**

In [32]:
# Reshape input data to meet the requirements of the LSTM model
x_train_reshaped = x_train.reshape((x_train.shape[0], 1, 1))
x_test_reshaped = x_test.reshape((x_test.shape[0], 1, 1))

# Define the model
model = Sequential()

# Add an LSTM layer with, for example, 50 units
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))

# Add a Dense layer for the output
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae', RootMeanSquaredError()])  # Add RMSE as a metric

# Train the model
history = model.fit(x_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(x_test_reshaped, y_test))

# Evaluate the model
loss, mae, rmse = model.evaluate(x_test_reshaped, y_test)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")
print(f"Test RMSE: {rmse}")

# Calculate RMSE separately using scikit-learn
y_pred = model.predict(x_test_reshaped)
rmse_sklearn = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE (using scikit-learn): {rmse_sklearn}")

# Print the training history (loss, MAE, and validation loss over epochs)
print("Training Loss:", history.history['loss'])
print("Training MAE:", history.history['mae'])
print("Validation Loss:", history.history['val_loss'])

Epoch 1/10
149/149 [==============================] - 3s 7ms/step - loss: 7673144.0000 - mae: 2180.7036 - root_mean_squared_error: 2770.0439 - val_loss: 1221936.7500 - val_mae: 914.4338 - val_root_mean_squared_error: 1105.4125
Epoch 2/10
149/149 [==============================] - 2s 11ms/step - loss: 224208.7969 - mae: 293.0548 - root_mean_squared_error: 473.5069 - val_loss: 1496.2482 - val_mae: 24.4582 - val_root_mean_squared_error: 38.6814
Epoch 3/10
149/149 [==============================] - 1s 9ms/step - loss: 1159.5637 - mae: 18.9435 - root_mean_squared_error: 34.0524 - val_loss: 1152.7091 - val_mae: 18.5227 - val_root_mean_squared_error: 33.9516
Epoch 4/10
149/149 [==============================] - 1s 9ms/step - loss: 1137.5736 - mae: 18.4170 - root_mean_squared_error: 33.7279 - val_loss: 1152.0315 - val_mae: 18.4880 - val_root_mean_squared_error: 33.9416
Epoch 5/10
149/149 [==============================] - 2s 11ms/step - loss: 1138.3732 - mae: 18.4629 - root_mean_squared_error:

**Predict Stock Price**

In [34]:
# Use the trained model to make predictions on the test set
y_pred = model.predict(x_test_reshaped)

# If you reshaped the input data, flatten the predictions for easier comparison
y_pred = y_pred.flatten()

# Create a DataFrame to compare the actual and predicted values
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results)

38/38 [==============================] - 0s 2ms/step
           Actual    Predicted
0      701.299988   700.997009
1     1298.880005  1297.965210
2     4505.689941  4524.511719
3     2800.969971  2813.416504
4     6155.410156  6122.766602
...           ...          ...
1185  5905.910156  5866.959961
1186  5996.439941  6030.774414
1187   771.989990   778.702393
1188  1353.209961  1354.538696
1189  6540.189941  6550.790039

[1190 rows x 2 columns]
